In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as spy
import math as m
from numpy.fft import fft, ifft, fftfreq
import time
import meep as mp

In [2]:
grid = 1
dpml =1
cavityd = 10
cell = (grid+dpml + cavityd)*2 

In [3]:
resolution = 50   
tsim = 1000
courant = 0.5 
dt = courant  / resolution
Nt = int(tsim/dt + 1)
tslice = np.linspace(0, tsim, Nt)
hb=1.05459*10**(-34)
c=3*10**8
epsilon = 8.854*10**(-12)
k= 1.38*10**(-23)
tau = 2*Nt*2*10**(-15)

##
omegal = 2 * np.pi / Nt * np.linspace(0,Nt-1,Nt) 
ml = np.random.normal(0,1,Nt)
nl = np.random.normal(0,1,Nt)

def Dnsqr(omega,T):
    x = np.abs(omega/T)
    if x > 1e-3:
        tmp = abs(omega)*hb / (np.exp(abs(omega)*hb/T/k)-1) / np.pi/c 
        output = 6*hb /T**2/k**2  * tmp *c
    else:
        output = 6 /np.pi/T * (1 - x/2 + x **2/12 -x**4/720) ## Taylor expansion
    
    output = np.sqrt(output)
    return output
tempe = 30000
dlsqr = np.empty(Nt)

for i in range(Nt):
     dlsqr[i] = Dnsqr(omegal[i],tempe) 
def rms(temperature):
    return((((2/(epsilon*6*hb*c))**0.5)*k*temperature))
delta = rms(30000)
fc = delta*(ml + 1j * nl) * dlsqr/tau**0.5

Ef = ifft(fc)

In [4]:
np.real(Ef[1])

0.4414923039401236

In [5]:


source = [mp.Source(mp.CustomSource(np.real(Ef)),mp.Ex,mp.Vector3(11)),
          mp.Source(mp.CustomSource(np.real(Ef)),mp.Ex,mp.Vector3(-11))]

In [6]:
sim = mp.Simulation(cell_size =mp.Vector3(0),
                    geometry = [],
                    sources = source,
                    resolution = 10,
                    boundary_layers = [mp.PML(dpml)]
                    )

In [ ]:
sim.run(until=0.1)

-----------
Initializing structure...
time for choose_chunkdivision = 3.00407e-05 s
Working in 2D dimensions.
Computational cell is 0.1 x 0.1 x 0 with resolution 10
time for set_epsilon = 4.79221e-05 s
